In [3]:
import glob
import os
import json
import jsonlines
from transformers import BertTokenizer
import torch.nn as nn
import torch
import os 
# HuggingFace
from transformers import BertTokenizer, BertModel, BertConfig 

In [2]:
data=[json.loads(l) for l in open("data_sets/proc/moviescope/train_sentences.jsonl")]

In [100]:
class Bert2Clf(nn.Module):
    def __init__(self, args):
        super(Bert2Clf, self).__init__()
        self.args = args
        bert = BertModel.from_pretrained("bert-base-uncased")
        self.bert_last_layer=bert.encoder.layer[11]
        self.pooler=bert.pooler
        self.bert = BertModel.from_pretrained(args.bert_model)
        self.dropout= nn.Dropout(args.dropout)
        self.clf = nn.Linear(args.hidden_sz, args.n_classes)

    def forward(self, txt1, mask1, segment1,txt2, mask2, segment2):
        bsz= txt1.shape[0]
        
        _, cls_1 = self.bert(
            input_ids=txt1,
            token_type_ids=segment1,
            attention_mask=mask1,
            return_dict=False,
        )
        cls_1=cls_1.reshape(bsz,1,self.args.hidden_sz)
        
        embedding_t2=self.bert.embeddings(input_ids=txt2,token_type_ids=segment2)
        embedinng_t2=torch.cat([cls_1,embedding_t2],dim=1)
        outputs=self.bert(inputs_embeds=embedinng_t2,attention_mask=mask2)
        cls_2 = outputs[1]
        cls_2=cls_2.reshape(bsz,1,self.args.hidden_sz)
        
        mask_id = torch.LongTensor([103])#.cuda()
        mask_id = mask_id.unsqueeze(0).expand(bsz, 1)
        mask_token_embeds = self.bert.embeddings(mask_id)
        
        
        x=torch.cat([mask_token_embeds,cls_1,cls_2],dim=1)
        
        
        sequence_output = self.bert_last_layer(x)[0]
        pooled_output = self.pooler(sequence_output)
        return self.clf(pooled_output)

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [35]:
index=0
input1=tokenizer(data[index]["sentences"][0], return_tensors="pt", padding='max_length',truncation=True)
sentence1=input1.input_ids
segment1=input1.token_type_ids
attmask1=input1.attention_mask


In [58]:
if len(data[index]["sentences"])<2:
    data[index]["sentences"].append(" ")
    
input2=tokenizer(data[index]["sentences"][1], return_tensors="pt", padding='max_length',truncation=True)
sentence2=input2.input_ids[:,1:]
segment2=input2.token_type_ids[:,1:]
attmask2=input2.attention_mask


In [38]:
import argparse
args = argparse.Namespace(
        bert_model="bert-base-uncased",
        model="mmbt",
        batch_sz=8,
        task_type="multilabel",
        max_seq_len=512,
        num_image_embeds=3,
        n_workers=2,
        dropout=0.1,
        hidden_sz=768,
        gradient_accumulation_steps=32,
        max_epochs=40,
        lr=1e-4,
        patience=5,
        lr_patience=2,
        warmup=0.1,
        freeze_img=3,
        freeze_txt=6,
        img_hidden_sz=2048,
        data_path="",
        task="",
        img_embed_pool_type="avg",
        n_classes=13,
        savedir="checkpoints2",
        name="mmbt"

    )

In [65]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [66]:
device

device(type='cpu')

In [98]:
bert2=BertClf(args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.we

In [99]:
bert2.to(device)

BertClf(
  (bert_last_layer): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (pooler): BertPooler(
    (dense): Linear(in_feat

In [101]:
x=bert2(sentence1,attmask1,segment1,sentence2,attmask2,segment2)

In [103]:
x[0]

tensor([-0.2206, -0.1784,  0.0488, -0.2400,  0.1333, -0.0218,  0.2631,  0.4458,
        -0.0086,  0.2876, -0.0874, -0.3861, -0.3197], grad_fn=<SelectBackward>)

In [5]:
bert=BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
bert_last_layer=bert.encoder.layer[11]

In [20]:
bert_last_layer

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)